In [ ]:
import cadquery as cq
from jupyter_cadquery.cadquery import show, Assembly, Part, Faces

**Avoid clean error**


In [ ]:
cq.occ_impl.shapes.Shape.clean = lambda x: x

# Converters to jupyter-cadquery

In [ ]:
import html
import numpy as np
import math

def rgb(assy):
    def b(x):
        return int(255*x)
    
    if assy.color is None:
        return "#aaa"
    rgb = assy.color.wrapped.GetRGB()
    return "#%02x%02x%02x" % (b(rgb.Red()), b(rgb.Green()), b(rgb.Blue()))

def convert(assy, loc=None):
    loc = assy.loc if loc is None else loc * assy.loc
    color = rgb(assy)
    parent = [Part(cq.Workplane(shape.located(loc)), "%s_%d" % (assy.name, i), color=color) for i, shape in enumerate(assy.shapes)]
    children = [convert(c, loc) for c in assy.children]
    return Assembly(parent + children, assy.name)

colors = [
    "#e41a1c","#377eb8","#4daf4a","#984ea3","#ff7f00","#ffff33","#a65628","#f781bf","#999999",
    "#8dd3c7","#ffffb3","#bebada","#fb8072","#80b1d3","#fdb462","#b3de69","#fccde5","#d9d9d9",
]

def show_assy(assy):
    show(convert(assy), axes=True, axes0=True)
    
def show_constraints(assy, qs):
    constraints = []
    objects = []
    cache = {}

    for i, q1q2 in enumerate(qs):
        parts = []

        kind = q1q2[-1]
        
        if len(q1q2) == 3:
            q1q2 = (
                (q1q2[0].split("@")[0], q1q2[0]), 
                (q1q2[1].split("@")[0], q1q2[1])
            )
        else:
            q1q2 = (q1q2[0:2], q1q2[2:4])
                    
        for q in q1q2:
            name, shape = q
            if name in cache:
                obj = cache[name]["obj"]
                loc = cache[name]["loc"]
            else:
                obj = assy.objects[name].obj
                loc = assy.objects[name].loc

                parent = assy.objects[name].parent
                while parent is not None:
                    loc = parent.loc * loc
                    parent = parent.parent

                cache[name] =  {"obj": obj, "loc": loc, "shape": shape}
            
                objects.append(
                    Part(cq.Workplane(obj.val().located(loc)), name=name, show_faces=False)
                )
            
            label = str(shape)
            if isinstance(shape, str):
                shape = assy._query(shape)[1]

            parts.append(
                Faces(
                    cq.Workplane(cq.Workplane(shape).val().located(loc)), 
                    name=html.escape(label), 
                    color=colors[i%len(colors)]
                )
            )
        constraints.append(Assembly(parts, "%s_%d" % (kind, i)))
        
    show(Assembly([Assembly(objects, "objects")] + constraints), axes=True, axes0=True)


def accuracy(assy, cs):

    def relocate(name, shape):
        a = assy.objects[name]
        loc = a.loc

        parent = a.parent
        while parent is not None:
            loc = parent.loc * loc
            parent = parent.parent

        label = str(shape)
        if isinstance(shape, str):
            shape = assy._query(shape)[1]
        return cq.Workplane(cq.Workplane(shape).val().located(loc))

    def coord(vertex):
        return np.array((vertex.X, vertex.Y, vertex.Z), dtype=float)

    def center(face):
        c = face.Center()
        return np.array((c.x, c.y, c.z))

    def normal(face):
        n = face.normalAt()
        return np.array((n.x, n.y, n.z))

    def print_metric(results):
        l = max([len(r[1]) for r in results])
        h = ("Constraint", "Normal-Dist", "Normal-Angle", "Point-Dist")
        print(f"{h[0]:{l+7}s} {h[1]:12s}  {h[2]:12s}  {h[3]:12s}")
        print("-" * (l+46))
        for kind, label, nrm_dist, nrm_angle, pnt_dist in results:
            metric = f"{kind:5s} {label:{l}s} "
            metric += " "*27 if nrm_dist is None else f"{nrm_dist:12.9f}  {nrm_angle:12.8}°"
            metric += " "*13 if pnt_dist is None else f"{pnt_dist:12.9f}"
            print(metric)
        
    results = []
    for i, q1q2 in enumerate(cs):
        parts = []

        kind = q1q2[-1]
        
        if len(q1q2) == 3:
            n_q1q2 = (
                (q1q2[0].split("@")[0], q1q2[0]), 
                (q1q2[1].split("@")[0], q1q2[1])
            )
            label = "%s - %s" % q1q2[:2]
        else:
            n_q1q2 = (q1q2[0:2], q1q2[2:4])
            label = "%s<%s> - %s<%s>" % (q1q2[0], q1q2[1].__class__.__name__, q1q2[2], q1q2[3].__class__.__name__)
        
        shape1 = relocate(*n_q1q2[0])
        shape2 = relocate(*n_q1q2[1])
        
        pnt_dist = None
        nrm_dist = None
        nrm_angle = None
        
        if kind in ["Point", "Plane"]:
            c1, c2 = center(shape1.val()), center(shape2.val())
            pnt_dist = np.linalg.norm(c1 - c2)
        if kind in ["Axis", "Plane"]:
            n1, n2 = normal(shape1.val()), normal(shape2.val())
            nrm_dist = np.linalg.norm(n1 + n2) # distance between n1 and -n2 since n1 and n2 point opposite
            c = np.dot(n1, -n2) / np.linalg.norm(n1) / np.linalg.norm(n2)
            nrm_angle = np.arccos(np.clip(c, -1, 1)) / math.pi * 180
            
        results.append((kind, label, nrm_dist, nrm_angle, pnt_dist))
        
    print_metric(results)

**Some helpers**

In [ ]:
def L(x,y,z):
    return cq.Location(cq.Vector(x, y, z))

def C(*c):
    return cq.Color(*c)

def center(assy, name):
    return cq.Vertex.makeVertex(*assy.objects[name].obj.val().Center().toTuple())

def query_all(assy, q):
    name, kind, arg = q.split("@")

    tmp = cq.Workplane()
    obj = assy.objects[name].obj

    if isinstance(obj, (cq.Workplane, cq.Shape)):
        tmp.add(obj)
        res = getattr(tmp, kind)(arg)

    return res.objects if isinstance(res.val(), cq.Shape) else None

# Bearing

## Objects

In [ ]:
def ring(inner_radius, outer_radius, width):
    ring = (cq.Workplane(origin=(0, 0, -width / 2))
        .circle(outer_radius).circle(inner_radius)
        .extrude(width)
    )
    return ring

tol = 0.05
ball_diam = 5

r1, r2, r3, r4 = 4, 6, 8, 10
r5 = (r3 + r2) / 2
inner_ring = ring(r1, r2, ball_diam)
outer_ring = ring(r3, r4, ball_diam)

torus = cq.CQ(cq.Solid.makeTorus(r5, ball_diam / 2 + tol))
ball = cq.Workplane().sphere(ball_diam / 2)

inner = inner_ring.cut(torus)
outer = outer_ring.cut(torus)

show(Assembly([Part(ball, "ball"), Part(inner, "inner"), Part(outer, "outer")], "bearing"))

**Assembly**

In [ ]:
def balls(i):
    return "ball_%d" % i

number_balls = 6

assy = cq.Assembly(outer, loc=L(0, 0, ball_diam/2), name="outer", color=C("orange"))
assy.add(inner, loc=L(20, 0, 0), name="inner", color=C("orange"))
for i in range(number_balls):
    assy.add(ball, loc=L(6*i, 20, 0), name=balls(i), color=C("black"))
    
# helpers
assy.add(cq.Workplane().circle(1).extrude(1), loc=L(0,-20,0), name="_center")
assy.add(cq.Workplane().polygon(number_balls, 2*r5).extrude(ball_diam/2), loc=L(20,-20,0), name="_points")

**Constraints**

In [ ]:
points = query_all(assy, "_points@vertices@>Z")

cs = [
    ("outer@faces@<Z", "_center@faces@>Z", "Plane"),
    ("inner@faces@<Z", "_center@faces@>Z", "Plane"),
    ("_points@faces@<Z", "_center@faces@>Z", "Plane"),
] + [("_points", points[i],  
      balls(i), center(assy, balls(i)),  
      "Point") 
     for i in range(number_balls)]

for c in cs:
    assy.constrain(*c)
    
show_constraints(assy, cs)

In [ ]:
show_assy(assy)

In [ ]:
assy.solve()
show_assy(assy)

In [ ]:
show_constraints(assy, cs)

In [ ]:
accuracy(assy, cs)

In [ ]:
i = 0
loc = assy.loc
for i in range(number_balls):
    o = assy.objects[balls(i)]
    v = o.shapes[0].located(loc * o.loc).Center()
    print("%7.4f, %7.4f, %7.4f" % (v.x, v.y, v.z))

In [ ]:
["%7.4f, %7.4f, %7.4f" % (v.X, v.Y, v.Z) for v in query_all(assy, "_points@vertices@>Z")]

# Some Tests

In [ ]:
def create():
    b1 = cq.Workplane().box(1, 1, 2).faces('>Z').edges('>X').chamfer(0.4)
    b2 = cq.Workplane().box(.1, 2, 1).faces('>Z').edges('>Y').chamfer(0.1)
    b3 = cq.Workplane().box(2, .1, .5).faces('>Z').edges('>X').chamfer(0.1)
    b4 = cq.Workplane().box(1, 1, .2).faces('>Y').edges('>X').chamfer(0.1)

    assy = cq.Assembly(b1, loc=L(1, 1, 0), name="TOP")
    assy2 = cq.Assembly(b2, name="SECOND")
    assy3 = cq.Assembly(b3, name="THIRD", color=C('orange'))

    assy.add(assy2, color=C("green"))
    assy.add(assy3)
    assy.add(b4, name="4th",color=C("blue1"))
    return assy

In [ ]:
assy = create()
cs = [
    ("TOP@faces@>(1,0,1)", "THIRD@faces@>Z", "Plane"), 
    ("TOP@faces@<Y", "SECOND@faces@<Y", "Axis"),
    ("THIRD@faces@<X", "SECOND@faces@>X", "Plane"),
    ("SECOND@faces@>Z", "THIRD@faces@<Z", "Axis"),
    ("4th@faces@>Z", "SECOND@faces@<Z", "Plane"),
    ("4th@faces@>Y", "TOP@faces@<Y", "Axis"),    
]
show_constraints(assy, cs)

In [ ]:
for c in cs:
    assy.constrain(*c)

assy.solve()
#show(convert(assy))
show_constraints(assy, cs)

In [ ]:
accuracy(assy, cs)

In [ ]:
assy = create()
cs = []
cs.append(("TOP@faces@>(1,0,1)", "THIRD@faces@>Z", "Axis"))
cs.append(("TOP@faces@>Y", "THIRD@faces@<Y", "Axis"))
cs.append(("TOP@vertices@>(1,1,2)", "THIRD@vertices@>(1,1,2)", "Point"))

for c in cs:
    assy.constrain(*c)

show_constraints(assy, cs)

In [ ]:
assy.solve()
show_constraints(assy, cs)

In [ ]:
accuracy(assy, cs)

In [ ]:
assy.constrain("TOP@faces@>(1,0,1)", "THIRD@faces@>Z", "Plane")
assy.constrain("TOP@faces@<Y", "SECOND@faces@<Y", "Axis")
assy.constrain("THIRD@faces@<X", "SECOND@faces@>X", "Plane")
assy.constrain("SECOND@faces@>Z", "THIRD@faces@<Z", "Axis")
assy.constrain("4th@faces@>Z", "SECOND@faces@<Z", "Plane")
assy.constrain("4th@faces@>Y", "TOP@faces@<Y", "Axis")
assy.solve()